In [33]:
import glob
import re
import pathlib
import json

In [34]:
def read_metadata_from_safetensors(filename):
    with open(filename, mode="rb") as file:
        metadata_len = file.read(8)
        metadata_len = int.from_bytes(metadata_len, "little")
        json_start = file.read(2)

        assert metadata_len > 2 and json_start in (b'{"', b"{'"), f"{filename} is not a safetensors file"

        res = {}

        try:
            json_data = json_start + file.read(metadata_len-2)
            json_obj = json.loads(json_data)
            for k, v in json_obj.get("__metadata__", {}).items():
                res[k] = v
                if isinstance(v, str) and v[0:1] == '{':
                    try:
                        res[k] = json.loads(v)
                    except Exception:
                        pass
        except Exception:
             Exception(f"Error reading metadata from file: {filename}")

        return res

In [35]:
files = glob.glob('../wildcards/**/*.txt', recursive=True) + glob.glob('../wildcards/**/*.yaml', recursive=True)

pattern = re.compile('<lora:(.+?):')

In [36]:
matches = []
for file in files:
    with open(file, 'r') as f:
        matches += pattern.findall(f.read())
matches = set(matches)

In [37]:
loras = glob.glob('../../../models/Lora/*.safetensors')

ss_names = []

lora_dict = {}

for lora in loras:
    fname = pathlib.Path(lora).stem
    try:
        ss_name = read_metadata_from_safetensors(lora)['ss_output_name']
    except:
        ss_name = fname

    if ss_name in lora_dict.keys():
        lora_dict[ss_name] += [fname]
    else:
        lora_dict[ss_name] = [fname]
    ss_names.append(ss_name)
ss_names = set(ss_names)

In [38]:
matches.difference(ss_names)

{'CHARACTER_Machamp',
 'CHARACTER_Machoke',
 'CrystalMommy',
 'Dmon ill-000045',
 'Elsa frozen',
 'Garuda',
 'POKEMON_MEOWSTIC_ FEMALE',
 'POKEMON_MEOWSTIC_MALE',
 'Princess_Elise_-_Sonic_06_Illustrious',
 'Riley_Andersen_A',
 'SerahFarron',
 'Shiva',
 'YunaPony',
 'bigrbear_Artist_Style_PonyXL',
 'dina',
 'dva academy illus-000040',
 'lugia-05',
 'madeline-v1.0'}

In [39]:
for x in lora_dict:
    if len(lora_dict[x]) > 1:
        print(x, lora_dict[x])

last ['Academydva-000035', 'bigrbear_Artist_Style_PonyXL', '[20th Century Studios] Predalien Xenomorph Character Illustrious', "[DAGASI (Sega Sonic)] Miles 'Tails' Prower Character IllustriousXL"]
Amaterasu_Dimwitdog_Version__Okami_Illustrious ['Amaterasu_Dimwitdog_Version_Okami_Illustrious', 'Amaterasu_Dimwitdog_Version__Okami_Illustrious']
MachopMachokeMachamp_Pokemon_PONY_XL ['CHARACTER_Machamp', 'CHARACTER_Machoke']
lora ['CrystalMommy', 'dina', 'Garuda', 'SerahFarron', 'Shiva', 'YunaPony']
None ['Dmon ill-000045', 'dva academy illus-000040', 'madeline-v1.0']
lugia ['lugia-04', 'lugia-05']
Mercy_Overwatch_Ultimate_Pack_PonyXL_LoRA ['Mercy_Overwatch_Ultimate_Pack_PonyXL_LoRA_epoch_13', 'Mercy_Overwatch__Ultimate_Pack_PonyXL_LoRA_epoch_13', 'Mercy__Overwatch__Ultimate_Pack_PonyXL_LoRA_epoch_13']
POKEMON_MEOWSTIC_ ['POKEMON_MEOWSTIC_ FEMALE', 'POKEMON_MEOWSTIC_MALE']
Princess_Elise_-_Sonic_06 ['Princess_Elise_-_Sonic_06', 'Princess_Elise_-_Sonic_06_Illustrious']
Riley_Andersen ['Riley

In [40]:
lora_dict

{'0040 metal tassel_v3_pony': ['0040 metal tassel_v3_pony-000003'],
 '0270 Slit nipple piercings_v1_pony': ['0270 Slit nipple piercings_v1_pony'],
 '0325 printed transparent cheongsam_v1_pony': ['0325 printed transparent cheongsam_v1_pony'],
 '0402 Breast exposure fun_v1_pony': ['0402 Breast exposure fun_v1_pony'],
 '0438 chunli costume_v1_pony': ['0438 chunli costume_v1_pony'],
 '0469 Stellar Blade (Skin Suit)_v3_pony': ['0469 Stellar Blade (Skin Suit)_v3_pony'],
 '0526 See-through rhinestone chain clothes_v1_pony': ['0526 See-through rhinestone chain clothes_v1_pony'],
 '0539 Fubuki fan costume_v1_pony': ['0539 Fubuki fan costume_v1_pony'],
 '0588 fragmented swimsuit_v1_pony': ['0588 fragmented swimsuit_v1_pony'],
 '0765 flower wedding_v1_pony': ['0765 flower wedding_v1_pony'],
 '0895 DVA fan clothing_v1_pony': ['0895 DVA fan clothing_v1_pony'],
 '0947 reverse virgin killer sweater_v1_pony': ['0947 reverse virgin killer sweater_v1_pony'],
 '20th_Century_Studios_Xenomorph_Character_Po